# Imports

In [1]:
import glob
from langchain.document_loaders import (UnstructuredWordDocumentLoader,
                                         UnstructuredPowerPointLoader,
                                         UnstructuredPDFLoader)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


# Create Vector Database

In [3]:
for file in glob.glob('../data/*.pdf'):
    print(file)

../data/2022 Q3 AAPL.pdf
../data/2023 Q3 AMZN.pdf
../data/2023 Q3 AAPL.pdf
../data/2023 Q1 NVDA.pdf
../data/2022 Q3 MSFT.pdf
../data/2023 Q2 AMZN.pdf
../data/2022 Q3 AMZN.pdf
../data/2023 Q2 AAPL.pdf
../data/2023 Q2 NVDA.pdf
../data/2023 Q3 INTC.pdf
../data/2022 Q3 NVDA.pdf
../data/2023 Q2 INTC.pdf
../data/2023 Q1 AMZN.pdf
../data/2023 Q1 INTC.pdf
../data/2023 Q1 AAPL.pdf
../data/2023 Q2 MSFT.pdf
../data/2023 Q3 NVDA.pdf
../data/2023 Q3 MSFT.pdf
../data/2022 Q3 INTC.pdf


In [4]:
docs = [ PyPDFLoader(file).load() for file in glob.glob('../data/*.pdf')]

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [5]:
doc_splits

[Document(metadata={'source': '../data/2022 Q3 AAPL.pdf', 'page': 0}, page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-Q\n(Mark One)\n☒  QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended June 25, 2022\nor\n☐  TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdictionof incorporation or organization) (I.R.S. Employer Identification No.)\nOne Apple Park Way\nCupertino, California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)'),
 Document(metadata={'source': '../data/2022 Q3 AAPL.pdf', 'page': 0}, page_content='One Apple Park Way\nCupertino, Califo

In [8]:
from langchain_ollama import OllamaEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Alibaba-NLP/gte-large-en-v1.5")

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [11]:
hfe = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-large-en-v1.5",
                            model_kwargs={"device": "cpu",
                                          "trust_remote_code": True},
                            encode_kwargs={"normalize_embeddings": True})

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits, # chunks of text
    persist_directory="../data/vdb",
    collection_metadata={"hnsw:space": "cosine"},
    embedding=hfe
)

In [ ]:
retriever = vectorstore.as_retriever()

In [12]:
### Retrieval Grader
from langchain_ollama import ChatOllama

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOllama(model="llama3.2:latest")

structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "is there any tmanyok in the documents ?"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [15]:
retrieval_grader = grade_prompt | structured_llm_grader
question = "is there any single information about apple in the documents ?"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [16]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")



# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

/home/azzedine/.local/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


I couldn't find any specific information about Apple in the documents. The retrieved context appears to be related to a company's (INTC) quarterly report, but it doesn't mention Apple at all.


In [17]:
FINANCE_API_KEY='cte32t9r01qt478kvedgcte32t9r01qt478kvee0'

In [2]:
!pip install finnhub-python

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import finnhub

# Setup client
finnhub_client = finnhub.Client(api_key=FINANCE_API_KEY)

# Company News
# Need to use _from instead of from to avoid conflict
print(finnhub_client.company_news('AAPL', _from="2024-12-12", to="2024-12-13"))

[{'category': 'company', 'datetime': 1734134056, 'headline': 'Best Way To Invest $100,000: Build An 8-Stock Portfolio For Growth And Value', 'id': 131977467, 'image': 'https://static.seekingalpha.com/cdn/s3/uploads/getty_images/1445810162/image_1445810162.jpg?io=getty-c-w1536', 'related': 'AAPL', 'source': 'SeekingAlpha', 'summary': 'Learn how to maximize returns by focusing on high-conviction stocks across diverse sectors and using proven investment strategies.', 'url': 'https://finnhub.io/api/news?id=55d1529df95faa211f30c69c3af5c9bc27914aaccba3736ed4c24b15134a2681'}, {'category': 'company', 'datetime': 1734132244, 'headline': 'Apple CEO Tim Cook to meet with Trump on Friday', 'id': 131978707, 'image': 'https://media.zenfs.com/en/reuters.com/64056b6fb5224383d905f9a8a3967ae9', 'related': 'AAPL', 'source': 'Yahoo', 'summary': "Apple CEO Tim Cook will meet Donald Trump on Friday night at the U.S. president-elect's Mar-a-Lago resort in Florida, a source familiar with the matter told Reute

In [19]:
from langchain_core.tools import tool

# Define a tool to fetch the finance news
@tool
def get_finance_news(company:str, start_date:str, end_date:str) -> str:
    """A tool that fetches finance news from the Finnhub API."""
    # Your existing API call logic here
    finnhub_client = finnhub.Client(api_key=FINANCE_API_KEY)
    
    response = finnhub_client.company_news(company, _from=start_date, to=end_date)
    summary = response[0]['summary']
    return summary

get_finance_news.invoke({"company":'AAPL', "start_date":'2024-12-12', "end_date":'2024-12-13'})


'Learn how to maximize returns by focusing on high-conviction stocks across diverse sectors and using proven investment strategies.'

-----

# Creation de l'agent simple (Agent + Tools)

In [20]:
from langchain_core.tools import tool


@tool
def get_finance_news(company:str, start_date:str, end_date:str) -> str:
    """A tool that fetches finance news from the Finnhub API."""
    # Your existing API call logic here
    finnhub_client = finnhub.Client(api_key=FINANCE_API_KEY)
    
    response = finnhub_client.company_news(company, _from=start_date, to=end_date)
    summary = response[0]['summary']
    return summary




tools = [get_finance_news]

llm_with_tools = llm.bind_tools(tools)

In [21]:
from langchain_core.messages import HumanMessage

query = "Use the provided tool in order to ouput the latest news, had in this question: What are the latest news of the company 'AAPL' between those exact values 2024-12-12 2024-12-13"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'get_finance_news', 'args': {'company': 'AAPL', 'end_date': '2024-12-13', 'start_date': '2024-12-12'}, 'id': 'a2740e88-b4fc-40d7-af16-8ea62f2476d7', 'type': 'tool_call'}]


In [22]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_finance_news":get_finance_news}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content="Use the provided tool in order to ouput the latest news, had in this question: What are the latest news of the company 'AAPL' between those exact values 2024-12-12 2024-12-13", additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-18T14:59:13.090985686Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'get_finance_news', 'arguments': {'company': 'AAPL', 'end_date': '2024-12-13', 'start_date': '2024-12-12'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1373126877, 'load_duration': 70571114, 'prompt_eval_count': 235, 'prompt_eval_duration': 106765000, 'eval_count': 44, 'eval_duration': 1150467000}, id='run-eb949431-3f41-4b2c-82bd-25dc04e50770-0', tool_calls=[{'name': 'get_finance_news', 'args': {'company': 'AAPL', 'end_date': '2024-12-13', 'start_date': '2024-12-12'}, 'id': 'a2740e88-b4fc-40d7-af16-8ea62f2476d

In [23]:
llm_with_tools.invoke(messages).content

"Based on the tool output, I've searched for the latest news about AAPL between December 12th and 13th, 2024. Here's a summary of the latest news:\n\n* On December 12th, 2024, Bloomberg reported that Apple Inc. is exploring ways to expand its services segment, which includes Apple Music, Apple TV+, and Apple Arcade. The company is reportedly looking into new features and content offerings to attract more customers.\n* On December 13th, 2024, Reuters stated that Apple's CEO Tim Cook is set to receive a $75 million bonus in 2025, as part of his compensation package. This news comes after the company reported record-breaking sales and profits in its latest quarter.\n\nPlease note that these are just examples of news articles and may not reflect the actual events or news about AAPL during this time period."

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

recommander_prompt_template = ChatPromptTemplate.from_template("""
You are a recommender system that suggests the best investement move to do based on the latest news of a company.
The goal is to provide a recommendation based on the latest news of the company.
The recommendation should be based on the latest news of the company.
News: {news}
Recommendation: """)

-------------------------------

# Multi Agent

In [ ]:
from typing import Annotated

from langchain_ollama import ChatOllama
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


llm = ChatOllama(model="llama3.2:latest")
# Modification: tell the LLM which tools it can call
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)